In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.9 MB/s 
     |████████████████████████████████| 1.6 MB 43.3 MB/s 


In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """


In [3]:
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

tf.__version__ is 2.9.2
tf.keras.__version__ is: 2.9.0


In [4]:
(X_train, y_train) , (X_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
X_train = X_train / 255
X_test = X_test / 255

In [6]:
X_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [7]:
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from kerastuner.tuners import RandomSearch

In [8]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    model.add(layers.Dense(units=hp.Int('units',
                                            min_value=32,
                                            max_value=512,
                                            step=128),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2,1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [9]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='project_dir',
    project_name='minist_keras_tunner')

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [11]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 2 Complete [00h 01m 23s]
val_accuracy: 0.9628000259399414

Best val_accuracy So Far: 0.9628000259399414
Total elapsed time: 00h 02m 47s


In [12]:
tuner.results_summary()

Results summary
Results in project_dir/minist_keras_tunner
Showing 10 best trials
Trial summary
Hyperparameters:
units: 288
learning_rate: 0.0001
Score: 0.9628000259399414
Trial summary
Hyperparameters:
units: 160
learning_rate: 0.0001
Score: 0.9551500082015991


In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 416)               326560    
                                                                 
 dense_1 (Dense)             (None, 10)                4170      
                                                                 
Total params: 330,730
Trainable params: 330,730
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    for i in range(hp.Int('layers', 2,6)):
        model.add(layers.Dense(units=hp.Int('units',min_value=50,max_value=100,step=10),activation=hp.Choice('act_'+str(i),['relu','sigmoid'])))
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    #model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile('adam','sparse_categorical_crossentropy',metrics=['accuracy'])              
    """model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2,1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])"""
    return model


In [ ]:
tuner2 = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1,
    directory='project_dir2',
    project_name='minist_keras_tunner2')

In [ ]:
tuner2.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 34s]
val_accuracy: 0.9757000207901001

Best val_accuracy So Far: 0.9761000275611877
Total elapsed time: 00h 02m 56s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner2.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 90)                70650     
_________________________________________________________________
dense_1 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_2 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_3 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_4 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                9

In [ ]:
import numpy as np
img=np.random.randint(2, size=(6, 6))
img

array([[1, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [1, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 1]])

In [ ]:
filter1=np.array(
    [
        [1,1,1],
        [0,0,0],
        [1,1,1]
    ]
)
filter1

array([[1, 1, 1],
       [0, 0, 0],
       [1, 1, 1]])

In [ ]:
filter2=np.array(
    [
        [1,0,1],
        [1,0,1],
        [1,0,1]
    ]
)
filter2

array([[1, 0, 1],
       [1, 0, 1],
       [1, 0, 1]])

In [ ]:
np.matmul(img[0:3,0:3],filter1)

array([[1, 1, 1],
       [2, 2, 2],
       [1, 1, 1]])

In [ ]:
np.matmul(img[0:3,0:3],filter2)

array([[1, 0, 1],
       [2, 0, 2],
       [2, 0, 2]])